<a href="https://colab.research.google.com/github/2001lida/PythonLession2/blob/hw_3/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B53.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd
import re

df = pd.read_csv('train.csv')
# 1
print(df.dtypes)
print("\n")

# 2
survival_summary = df.groupby('Pclass').agg(
    total_passengers=('Survived', 'count'),
    survived=('Survived', 'sum'),
    survival_rate=('Survived', 'mean')
).round(4)
survival_summary['survival_rate_%'] = (survival_summary['survival_rate'] * 100).round(2).astype(str) + '%'

print(survival_summary)
print("\n")

# 3
# Функция для извлечения личного имени
def extract_first_name(name):
    # Убираем фамилию (всё до первой запятой)
    if ',' in name:
        name = name.split(',', 1)[1].strip()

    # Ищем имя в скобках (девичье имя у замужних женщин)
    bracket_match = re.search(r'\(([^)]+)\)', name)
    if bracket_match:
        name = bracket_match.group(1).strip()

    # Убираем обращения (Mr., Mrs., Miss., Master. и т. п.)
    name = re.sub(r'(Mr\.|Mrs\.|Miss\.|Master\.|Dr\.|Rev\.|Col\.|Major\.|Capt\.)\s*', '', name)

    # Берём первое слово (основное имя) и убираем лишние символы
    name = name.split()[0].strip('"\'()')

    return name

# Применяем функцию к столбцу Name
df['First_Name'] = df['Name'].apply(extract_first_name)

# Разделяем по полу и находим самое популярное имя
male_names = df[df['Sex'] == 'male']['First_Name']
female_names = df[df['Sex'] == 'female']['First_Name']

most_common_male = male_names.value_counts().idxmax()
male_count = male_names.value_counts()
most_common_female = female_names.value_counts().idxmax()
female_count = female_names.value_counts()

# Считаем частоты
female_name_counts = female_names.value_counts()
male_name_counts = male_names.value_counts()

# Находим максимальное количество встреч
female_max_count = female_name_counts.max()
male_max_count = male_name_counts.max()

female_most_popular = female_name_counts[female_name_counts == female_max_count]
male_most_popular = male_name_counts[male_name_counts == male_max_count]

# Выводим результат
print(f"Самые популярные женские имена ({female_max_count} раз):")
for name, count in female_most_popular.items():
    print(f"  {name} — {count} раз")

print(f"Самые популярные мужские имена ({male_max_count} раз):")
for name, count in male_most_popular.items():
    print(f"  {name} — {count} раз")

print("\n")

# 4

df['First_Name'] = df['Name'].apply(extract_first_name)

# Приводим к единому регистру
df['First_Name'] = df['First_Name'].str.capitalize()

# Удаляем пропуски
df = df.dropna(subset=['Pclass', 'Sex', 'First_Name'])

def get_most_popular(group):
    counts = group['First_Name'].value_counts()
    max_count = counts.max()
    most_popular = counts[counts == max_count]
    result = most_popular.reset_index()
    result.columns = ['Имя', 'Кол-во']

    return result

result = ( df.groupby(['Pclass', 'Sex'])
    .apply(get_most_popular, include_groups=False)
    .reset_index(level=[0,1]) )

result = result.rename(columns={
    'Pclass': 'Класс',
    'Sex': 'Пол'
})

if isinstance(result.columns, pd.MultiIndex):
    result.columns = result.columns.droplevel(0)

result = result.reset_index(drop=True)

print("Самые популярные имена по классам и полу:")
print(result)
print("\n")

# 5
adults_subset = df[df['Age'] > 44][['Name', 'Age', 'Pclass', 'Sex']]

print("Пассажиры старше 44 лет (имя, возраст, класс, пол):")
print(adults_subset)
print("\n")

# 6
young_man_subset = df[(df['Age'] < 44) & (df['Sex'] == 'male')][['Name', 'Age', 'Pclass']]

print("Пассажиры младше 44 лет (имя, возраст, класс):")
print(young_man_subset)
print("\n")

# 7
df = df.dropna(subset=[ 'Ticket' ])

ticket_counts = df['Ticket'].value_counts()

df = df[ df['Ticket'].map(ticket_counts) > 1 ]

# Сколько пассажиров на каждый Ticket
ticket_counts = df['Ticket'].value_counts()

group_size_counts = ticket_counts.value_counts().sort_index()

result = (
    group_size_counts
    .rename_axis('Число пассажиров в группе')
    .reset_index(name='Количество билетов')
)

print("Количество n-местных групп (по Ticket):")
print(result)




PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


        total_passengers  survived  survival_rate survival_rate_%
Pclass                                                           
1                    216       136         0.6296          62.96%
2                    184        87         0.4728          47.28%
3                    491       119         0.2424          24.24%


Самые популярные женские имена (15 раз):
  Anna — 15 раз
  Mary — 15 раз
Самые популярные мужские имена (36 раз):
  William — 36 раз


Самые популярные имена по классам и полу:
   Класс     Пол        Имя  Кол-во
0      1  female  Elizabeth       5
1      1  female   Margaret       5
2      1    male    William      11
3      2  female  Elizabeth       5
4      2    male   